# PA2.2 Part B - Hidden Markov Model (HMM) Based Music Generator

### Introduction

In this notebook, you will be generating Music (no vocals though) using an HMM via [Baum-Welch](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) Training Algorithm.

In the context of Music Generation, the states might represent underlying musical concepts (like note pitches or chord types), and observations could be specific notes or chords played at a given time. Hence, generating music involves moving through the states based on transition probabilities and producing musical notes based on the emission probabilities associated with each state. The emission probabilities dictate how likely it is to observe each possible note or chord (observation symbol) when in a given state.

## Terminology

__Baum Welch__: is an __*Unsupervised*__ training algorithm that involves adjusting the HMM's parameters (transition, emission, and initial state probabilities) to best account for the observed sequences.The training process involves:
- Expectation step (E-step): Estimate the likely sequences of hidden states (could be something implicit like musical concepts like chords or rhythms) given the current parameters of the model and the observed data.
- Maximization step (M-step): Update the model's parameters to maximize the likelihood of the observed data, based on the estimated sequences of hidden states.

![Baum Welch](unsupervised_learning.png)

## Resources

For additional details of the working of Baum-Welch Training you can consult these medium articles [Baum-Welch algorithm](https://medium.com/mlearning-ai/baum-welch-algorithm-4d4514cf9dbe) and [Baum-Welch algorithm for training a Hidden Markov Model — Part 2 of the HMM series](https://medium.com/analytics-vidhya/baum-welch-algorithm-for-training-a-hidden-markov-model-part-2-of-the-hmm-series-d0e393b4fb86) as reference.

A more technical overview is covered by Rabiner in his paper on [A Tutorial on Hidden Markov Models and Selected Applications in Speech Recognition](http://www.stat.columbia.edu/~liam/teaching/neurostat-fall17/papers/hmm/rabiner.pdf).

If the above link is a bit difficult to digest, you can consult the following slides by Stanford's Dan Jurafsky in his course [LSA 352: Speech Recognition and Synthesis](https://nlp.stanford.edu/courses/lsa352/lsa352.lec7.6up.pdf).

### Instructions

- Follow along with the notebook, filling out the necessary code where instructed.

- <span style="color: red;">Read the Submission Instructions, Plagiarism Policy, and Late Days Policy in the attached PDF.</span>

- <span style="color: red;">Make sure to run all cells for credit.</span>

- <span style="color: red;">Do not remove any pre-written code.</span>

- <span style="color: red;">You must attempt all parts.</span>

For this notebook, in addition to standard libraries i.e. `numpy`, `tqdm`, `hmmlearn` and `muspy`, you are permitted to incorporate supplementary libraries, but it is strongly advised to restrict their inclusion to a minimum. However, other HMM toolkits or libraries are strictly prohibited.

In [1]:
!pip install muspy
!pip install hmmlearn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=9b7f3cc693e681e58d9b1d5468d9ba5bff493aad35476a154f63f6503bebf37b
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty-midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 1.9 MB/s eta 0:00:00


**MusPy** is an open source Python library for symbolic music generation. It provides essential tools for developing a music generation system, including dataset management, data I/O,  
data preprocessing and model evaluation.  
**Documentation**: https://salu133445.github.io/muspy/

In [2]:
import numpy as np
from tqdm.notebook import tqdm as tqdm
import muspy
from hmmlearn.hmm import CategoricalHMM
muspy.download_musescore_soundfont()

Start downloading MuseScore General soundfont.
MuseScore General soundfont has successfully been downloaded to : /root/.muspy/musescore-general.


In [3]:
#List of available datasets in muspy (could also add your own datasets)
#Link: https://salu133445.github.io/muspy/datasets/datasets.html
print(muspy.list_datasets())

[<class 'muspy.datasets.emopia.EMOPIADataset'>, <class 'muspy.datasets.essen.EssenFolkSongDatabase'>, <class 'muspy.datasets.haydn.HaydnOp20Dataset'>, <class 'muspy.datasets.hymnal.HymnalDataset'>, <class 'muspy.datasets.hymnal.HymnalTuneDataset'>, <class 'muspy.datasets.jsb.JSBChoralesDataset'>, <class 'muspy.datasets.lmd.LakhMIDIAlignedDataset'>, <class 'muspy.datasets.lmd.LakhMIDIDataset'>, <class 'muspy.datasets.lmd.LakhMIDIMatchedDataset'>, <class 'muspy.datasets.maestro.MAESTRODatasetV1'>, <class 'muspy.datasets.maestro.MAESTRODatasetV2'>, <class 'muspy.datasets.maestro.MAESTRODatasetV3'>, <class 'muspy.datasets.music21.Music21Dataset'>, <class 'muspy.datasets.musicnet.MusicNetDataset'>, <class 'muspy.datasets.nes.NESMusicDatabase'>, <class 'muspy.datasets.nmd.NottinghamDatabase'>, <class 'muspy.datasets.wikifonia.WikifoniaDataset'>]


Since Baum-Welch is known for its slow convergence, we'll take the lightest dataset available from muspy datasets called the __HaydnOp20__ Dataset consisting of 1.26 hours of recordings \
comprising of 24 classical songs

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
my_dataset = muspy.datasets.HaydnOp20Dataset(root = '/content/drive/MyDrive/GenAi/PA2.2_dataset/data', download_and_extract=True)
my_dataset = my_dataset.convert()

Skip downloading as the `.muspy.success` file is found.
Skip extracting as the `.muspy.success` file is found.
Skip conversion as the `.muspy.success` file is found.


## Section 1: (HMM + Baum Welch From Scratch) [80 Marks]

Muspy offers all datasets in 4 different representations as mentioned below:

![Alt text](muspy_representations.png)

Initially, we are only interested in modelling through time and to keep it simple, we'll begin with the __Pitch Representation__. More details here:

![text](muspy_to_pitch.png)

In [ ]:
music_collection = []
for music in my_dataset:
    music_collection.append(muspy.to_pitch_representation(music, use_hold_state=True))

### Singing HMM Class
The __Singing_HMM__ Class contains the following methods:

1. `__init__(self, corpus)`: initializes the __POS_HMM__ and prepares it for the parameter initialization phase, contains:
    - a corpus, consisting of unlabeled  sequences of musical units (i.e. all the music songs are flattened and concatenated)
    - a hidden_state_size (default to 10), higher values capture more variability but converge slowly.
    - a tuple of all the unique
    - a dictionary for mapping the pitches to its unique integer identifier.
    - some additional variables to reduce code redundancy in latter parts such as len()
    - Transition, Emission and Initial State Probability Matrices which are initialized to Zeros.

2. `init_mat(self, init_scheme='uniform')`: __(Can Be Modified)__ initializes the transition, emission and probability matrices either with a 'uniform' value or values sampled randomly from a uniform distribution and normalizes the matrice row wise.

3. `forward(self, sequence)`: __(To Be Implemented)__ implements the Forward stage of the Forward-Backward Algorithm.
- Feel free to modify function signature and return values.
- Do not change the function name.
4. `backward(self, sequence)`: __(To Be Implemented)__ implements the Forward stage of the Forward-Backward Algorithm.
- Feel free to modify function signature and return values.
- Do not change the function name.
6. `baum_welch(sequence, alpha, beta)`: __(To Be Implemented)__ implements the Baum Welch Training Algorithm.
- Feel free to modify function signature and return values.
- Do not change the function name.
7. `softmax(self, x, temperature=1.0)`: calculates the softmax of a given input x adjusting the sharpness of the distribution based on a temperature parameter.

8. `temperature_choice(self, probabilities, temperature=1.0)`: applies a temperature scaling to a set of probabilities and selects an index based on the adjusted probabilities.

9. `sample_sequence(self, length, strategy = "temperature", temperature = 1.0)`: __(Can Be Modified)__ generates a sequence of elements based on a given strategy (probabilistic or temperature) and a specified length. Strategies consists of:
* `probabilistic` strategy:
    -  Samples the initial state based on initial state probabilities.
    -  Iterates over the desired sequence length, sampling an observation based on the current state's emission probabilities, appending the observation to the sequence, and then transitioning to the next state based on the current state's transition probabilities.
* `temperature` strategy:
    -  Similar to the probabilistic strategy but applies temperature scaling to the choice of initial state, observation sampling, and state transitions to adjust the randomness of the choices.

##### __READ THIS BEFORE YOU BEGIN__:
- The functions `init_mat` and  `sample_sequence` are although pre-defined and will work properly, but if you have a better strategy feel free to add or experiment. Just make sure not to overwrite the pre-existing code.
- You are allowed to make helper functions, just make sure they are neatly structured i.e. have self-explanatory variable names, have an algorithmic flow to them and are sufficiently commented.
- Make sure not to change any exisiting code unless allowed by the TA.

__Tips for Baum-Welch Implementation__:

1. Write the code for simple/vanilla Baum Welch Implementation first.
2. You have the option to either go over the whole concatenated sequence or each music seperately (in a nested for loop) per iteration.
3. If your vanilla Baum Welch Implementation compiles, most likely you would get overflow errors, arising from division by 0. This is due to long sequences yielding  \
smaller values of alpha and beta. Hence, wherever division occurs, the denominator variable (which is a result of multiplication with alpha or beta) is close to 0.

I'll now suggest some ways with which the third point can be alleviated __(the hacky ways might/might not work, so be wary)__:

- Hacky way #1 (Working with smaller chunks of observed sequences): For every iteration, rather than going over the concatenated music sequences or each music sequence, you can further break down your musical sequences into even smaller chunks and go over those instead.

- Hacky way #2 (Add a small epsilon value to the denominator): Add a small episilon value like 1e-12 to the denominator wherever the division by 0 error occurs.

- Proper way #1 (The [log-sum-exp](https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/) trick): For an HMM, the smaller values can be dealt with by passing them through
    log and converting the multiplications to additions and then brought back via exponentiating them.

    - Another [intro](https://www.xarg.org/2016/06/the-log-sum-exp-trick-in-machine-learning/) for the log-sum-exp, if the previous one was unclear.
    - [Hidden Markov Models By Herman Kemper](https://www.kamperh.com/nlp817/notes/05_hmm_notes.pdf) illustrates the use of log-sum-exp technique in Baum Welch Implementation (particularly Forward and Backward Passes).
    - [Recition 7: Hidden Markov Models](https://www.cs.cmu.edu/~mgormley/courses/10601-s23/handouts/hw7_recitation_solution.pdf) gives an idea of the usage of log-sum-exp in the forward-backward algorithm.
    - This HMM github [repo](https://github.com/jwmi/HMM/blob/master/hmm.jl) has implemented the log-sum-exp trick in julia language.
    - The following [blog post](https://gregorygundersen.com/blog/2020/11/28/hmms/#implementation) might also be helpful for implementation of baum-welch using log-sum-exp trick.
    - The following paper on [Numerically Stable Hidden Markov Models](http://bozeman.genome.washington.edu/compbio/mbt599_2006/hmm_scaling_revised.pdf) gives pseudocodes for working in the log domain for the HMMs (although not necessarily the log-sum-exp trick as is).

- Proper way #2 (Scaling Factors): involves scaling the alpha and beta values to avoid underflows.
    - The following blog post explains the maths behind scaling [Scaling Factors for Hidden Markov Models](https://gregorygundersen.com/blog/2022/08/13/hmm-scaling-factors/)
    - This stackexchange post [Scaling step in Baum-Welch algorithm](https://stats.stackexchange.com/questions/274175/scaling-step-in-baum-welch-algorithm) contains two answers which can also be consulted.

__How do you know the HMM is converging?__:

Since Baum Welch algorithm guarantees convergence to the local (not global) maxima, near zero values are difficult to achieve.  \
Hence, a convergening HMM would have the log likelihoods going towards 0 (although still far from it). You can find a sample cell output  \
below showing the log likelihoods decreasing. Another way is to see is that the post-convergence generated music would be better than the  \
starting HMM (which has uniform or randomly initialized matrices).

__How do you know the HMM has converged?__:

One way is to monitor the difference between two successive log likelihoods and stop when the differences goes below a certain threshold. This has already been implemented for you.

In [ ]:
class Singing_HMM:
    def __init__(self, corpus, hidden_state_size=10):
        self.corpus = [seq.flatten().tolist() for seq in corpus]
        self.hidden_state_size = hidden_state_size
        self.music_seq = [note for seq in self.corpus for note in seq]
        self.vocab = tuple(set(self.music_seq))
        self.vocab2index = {note: i for i, note in enumerate(self.vocab)}
        self.vocab_len = len(self.vocab)

        self.transition_mat = np.zeros((self.hidden_state_size, self.hidden_state_size))
        self.emission_mat = np.zeros((self.hidden_state_size, self.vocab_len))
        self.initial_state_prob = np.zeros(self.hidden_state_size)

        #----------------Add Any Additional Code Below This Line----------------

    @staticmethod
    def log_sum_exp(x, axis=None):
      if x.ndim > 1:
        x_max = np.max(x, axis=axis, keepdims=True)
        return x_max + np.log(np.sum(np.exp(x - x_max), axis=axis, keepdims=True))
      else:
        x_max = np.max(x)
        return x_max + np.log(np.sum(np.exp(x - x_max)))



    #Feel free to define any helper functions

    def init_mat(self, init_scheme='uniform'): # Can be optionally modified for another initialization scheme (not necessary for the assignment)
        if init_scheme == 'uniform':
            self.transition_mat = np.ones((self.hidden_state_size, self.hidden_state_size))
            self.emission_mat = np.ones((self.hidden_state_size, self.vocab_len))
            self.initial_state_prob = np.ones(self.hidden_state_size)
        elif init_scheme == 'random':
            self.transition_mat = np.random.rand(self.hidden_state_size, self.hidden_state_size)
            self.emission_mat = np.random.rand(self.hidden_state_size, self.vocab_len)
            self.initial_state_prob = np.random.rand(self.hidden_state_size)

        self.transition_mat /= self.transition_mat.sum(axis=1, keepdims=True)
        self.emission_mat /= self.emission_mat.sum(axis=1, keepdims=True)
        self.initial_state_prob /= self.initial_state_prob.sum()

    def forward(self, sequence):
        # log_sum_values = np.zeros((len(sequence) - 1, self.hidden_state_size))
        """
        Forward algorithm for calculating the probabilities of a sequence.
        """
        log_alpha = np.zeros((len(sequence), self.hidden_state_size))
        log_alpha[0, :] = self.initial_state_prob * self.emission_mat[:, self.vocab2index[sequence[0]]]

        for t in range(1, len(sequence)):
            for j in range(self.hidden_state_size):
              log_sum = np.zeros(self.hidden_state_size)
              for i in range(self.hidden_state_size):
                log_sum[i] = log_alpha[t - 1, i] + np.log(self.transition_mat[i, j])
                log_alpha[t, j] = self.log_sum_exp(log_sum) + np.log(self.emission_mat[j, self.vocab2index[sequence[t]]])
              # log_sum_values[t-1]=log_sum


                #alpha[t, j] = alpha[t-1].dot(self.transition_mat[:, j]) * self.emission_mat[j, self.vocab2index[sequence[t]]]


        return log_alpha

    def backward(self, sequence):
        # log_sum_values = np.zeros((len(sequence) - 1, self.hidden_state_size))  # Initialize 2D array to store log_sum[i] values

        """
        Backward algorithm for calculating the probabilities of a sequence.
        """
        log_beta = np.zeros((len(sequence), self.hidden_state_size))
        log_beta[len(sequence) - 1] = 0

        for t in range(len(sequence) - 2, -1, -1):
          for i in range(self.hidden_state_size):
            log_sum = np.zeros(self.hidden_state_size)
            for j in range(self.hidden_state_size):
              log_sum[j] = np.log(self.transition_mat[i, j]) + \
                         np.log(self.emission_mat[j, self.vocab2index[sequence[t + 1]]]) + \
                         log_beta[t + 1, j]
            # log_sum_values[t-1] = log_sum
              log_beta[t, i] = self.log_sum_exp(log_sum)

              #beta[t, i] = np.sum(beta[t+1] * self.transition_mat[i, :] * self.emission_mat[:, self.vocab2index[sequence[t+1]]])

        return log_beta

    def baum_welch(self, n_iter=100, tol=1e-4):
        """
        Perform Baum-Welch training to update the model's parameters.
        """

        prev_log_likelihood = float('-inf')  # Initialize with negative infinity (DO NOT CHANGE THIS VARIABLE)

        for iteration in tqdm(range(n_iter), desc="Training Progress", leave=True):
            log_likelihood = 0 # Log likelihood for this iteration (DO NOT CHANGE THIS VARIABLE)
            #----------------Add Your Code Here----------------
            A_new = np.zeros_like(self.transition_mat)
            B_new = np.zeros_like(self.emission_mat)
            Pi_new = np.zeros_like(self.initial_state_prob)
            for sequence in self.corpus:
              alpha = self.forward(sequence)
              beta = self.backward(sequence)

              # Calculating sequence log likelihood
              log_likelihood_seq = self.log_sum_exp(alpha[-1])
              log_likelihood += log_likelihood_seq

              # Calculating gamma and xi for the sequence
              gamma = np.zeros((len(sequence), self.hidden_state_size))
              xi = np.zeros((len(sequence) - 1, self.hidden_state_size, self.hidden_state_size))

              for t in range(len(sequence)):
                log_gamma_t = alpha[t] + beta[t] - self.log_sum_exp(alpha[t] + beta[t], axis=1)
                gamma[t] = np.exp(log_gamma_t)  # Converting back to probability space for updating params

              for x in range(len(sequence) - 1):
                for y in range(self.hidden_state_size):
                  for z in range(self.hidden_state_size):
                    alpha_value = alpha[x, y]
                    transition_log_prob = np.log(self.transition_mat[y, z])
                    emission_log_prob = np.log(self.emission_mat[z, self.vocab2index[sequence[x+1]]])
                    beta_value = beta[x+1, z]
                    log_xi = alpha_value + transition_log_prob + emission_log_prob + beta_value - log_likelihood_seq
                    xi[x, y, z] = np.exp(log_xi)  # Converting back to probability space for updating params


              A_new += np.sum(xi, axis=0)
              for t in range(len(sequence)):
                B_new[:, self.vocab2index[sequence[t]]] += gamma[t]

                # Updating initial state distribution
              Pi_new += gamma[0]

                # Normalizing A, B, Pi to get the new model parameters
            self.transition_mat = A_new / np.sum(A_new, axis=1, keepdims=True)
            self.emission_mat = B_new / np.sum(B_new, axis=1, keepdims=True)
            self.initial_state_prob = Pi_new / np.sum(Pi_new)



















            #----------------Do Not Modify The Code Below This Line----------------

            if iteration == 0:
                convergence_rate = convergence_diff = np.nan  # Print nan for the first iteration
            else:
                convergence_diff = np.abs(log_likelihood - prev_log_likelihood)
                convergence_rate = convergence_diff / np.abs(prev_log_likelihood)

            #Note that Log Likelihoods would be negative and would increase (i.e. go in the direction of 0) as the model converges.
            # Log Likelihoods may be far from 0, but the increasing trend should remain present.
            tqdm.write(f"Iteration {iteration + 1}: Log Likelihood: {log_likelihood}, Convergence Difference: {convergence_diff} , Convergence Rate: {convergence_rate}")

            if iteration > 0 and convergence_rate < tol:
                tqdm.write("Convergence achieved.")
                break

            prev_log_likelihood = log_likelihood

    def softmax(self, x, temperature=1.0):
        '''Compute softmax values for each set of scores in x.'''
        e_x = np.exp((x - np.max(x)) / temperature)
        return e_x / e_x.sum()

    def temperature_choice(self, probabilities, temperature=1.0):
        '''Apply temperature to probabilities and make a choice.'''
        adjusted_probs = self.softmax(np.log(probabilities + 1e-9), temperature)  # Adding epsilon to avoid log(0)
        return np.random.choice(len(probabilities), p=adjusted_probs)

    def sample_sequence(self, length, strategy = "temperature", temperature = 1.0):
        sequence = []
        if strategy == 'probabilistic':
            # Sample the initial state
            state = np.random.choice(self.hidden_state_size, p=self.initial_state_prob)
            for _ in range(length):
                # Sample an observation (note) based on the current state
                note = np.random.choice(self.vocab, p=self.emission_mat[state])
                sequence.append(note)
                # Transition to the next state
                state = np.random.choice(self.hidden_state_size, p=self.transition_mat[state])
        elif strategy == 'temperature':
            # Sample the initial state with temperature
            state = self.temperature_choice(self.initial_state_prob, temperature)
            for _ in range(length):
                # Apply temperature to emission probabilities and sample a note
                note = self.temperature_choice(self.emission_mat[state], temperature)
                sequence.append(self.vocab[note])
                # Transition to the next state with temperature
                state = self.temperature_choice(self.transition_mat[state], temperature)
        return sequence

In [ ]:
#Specify values and run the code to test your implementation
pos_hmm = Singing_HMM(corpus = music_collection, hidden_state_size =10)
pos_hmm.init_mat(init_scheme = 'random') #Note: HMMs are sensitive to intialization schemes
pos_hmm.baum_welch(tol = 1e-4, n_iter= 10)

Training Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration 1: Log Likelihood: -477879.25164318783, Convergence Difference: nan , Convergence Rate: nan
Iteration 2: Log Likelihood: -98308.30500076982, Convergence Difference: 379570.94664241804 , Convergence Rate: 0.7942821232293792
Iteration 3: Log Likelihood: -97530.83774709034, Convergence Difference: 777.4672536794824 , Convergence Rate: 0.007908459551544443
Iteration 4: Log Likelihood: -93685.03601827337, Convergence Difference: 3845.801728816965 , Convergence Rate: 0.039431648672900874
Iteration 5: Log Likelihood: -86792.14462301758, Convergence Difference: 6892.891395255792 , Convergence Rate: 0.07357515872557627
Iteration 6: Log Likelihood: -82256.12173016275, Convergence Difference: 4536.022892854831 , Convergence Rate: 0.05226305805159079
Iteration 7: Log Likelihood: -80606.38729283433, Convergence Difference: 1649.734437328414 , Convergence Rate: 0.020056068808353117
Iteration 8: Log Likelihood: -80417.50744573558, Convergence Difference: 188.8798470987531 , Convergence Rate

In [ ]:
notes_seq = pos_hmm.sample_sequence(1024, strategy= "probabilistic") #Feel free to experiment with the sampling strategy
synthetic_music = muspy.from_pitch_representation(np.array(notes_seq), resolution=24, program=0, is_drum=False, use_hold_state=True, default_velocity=64)
muspy.write_midi('/content/drive/MyDrive/GenAi/PA2.2_dataset/pitch_based.midi', synthetic_music) #Specify the path to save the MIDI file, name it "pitch_based.mid"

You can visualize your results here: https://cifkao.github.io/html-midi-player/  
  
Remember to brag about your generated music on Slack (You can use online __MIDI to WAV/MP3__ Converters)

__*P.S*__: You can use muspy.write_audio to convert the music object directly to wav file but that requires installation of a few softwares (not worth the hassle).

*You might notice that the results are although better than random but they are not as awe-inspiring as intended.
The reason being that our model is unable to capture the  \
variability of the different music styles (our dataset comprises of). However, there is a way to generate better music, that is taking a sufficiently long MIDI  \
(could be other formats as well) sound track(s) of a single artist (EDM or any music which has repetitiveness in it) and refitting your HMM.  \
The relevent function here would be muspy.read_midi().  \
__After training on the notebook provided dataset, you are more than welcome to try it with your own curated dataset and see the results.  \
THIS IS OPTIONAL AND NOT MANDATORY__.*

## Section 2: Synthetic Music Generation via __HMMLearn__ [20 Marks]

#### __Note:__ For any model that you train/fit, remember to set __verbose = True__

In [6]:
hidden_states = 32 #Number of hidden states in the HMM model (Feel free to change or experiment with this value)
synthetic_music_sequence_length = 128 #Length of the synthetic music sequence to be generated (could be either a time step, an event or a note)

For starters, let's replicate what we did above manually with our HMM Library. Since, we already did pitch based representation,  
let's do it for **Event Based Representation** (which is essentially denotes music as a sequence of events). So while pitch based representation  
is between 0-128 unique pitch values, the event based representation is between 0-387 unique events.

In [7]:
import numpy as np
from hmmlearn import hmm

music_collection = []
for music in my_dataset:
    music_collection.append(muspy.to_event_representation(
        music,
        use_single_note_off_event=False,
        use_end_of_sequence_event=False,
        encode_velocity=False,
        force_velocity_event=True,
        max_time_shift=100,
        velocity_bins=32,
        dtype=int  # Note: <class 'int'> is replaced with int
    ))


In [8]:
lengths = [len(sequence) for sequence in music_collection]
X = np.concatenate(music_collection).reshape(-1, 1)  # Reshaping for hmmlearn



model = hmm.CategoricalHMM(n_components=hidden_states, n_iter=100, random_state=42,verbose=True)
model.fit(X, lengths)

generated_sequence, _ = model.sample(synthetic_music_sequence_length)


         1 -341041.20852393             +nan
         2 -246409.37077765  +94631.83774628
         3 -242343.56716340   +4065.80361425
         4 -235623.65683487   +6719.91032853
         5 -228631.36032095   +6992.29651393
         6 -221798.95378898   +6832.40653196
         7 -215848.67193660   +5950.28185238
         8 -211472.36904498   +4376.30289162
         9 -208091.04015775   +3381.32888724
        10 -205308.27993978   +2782.76021796
        11 -203145.25062401   +2163.02931578
        12 -201498.31234618   +1646.93827783
        13 -200233.01066918   +1265.30167700
        14 -199203.80398059   +1029.20668859
        15 -198285.35112048    +918.45286011
        16 -197679.42038016    +605.93074032
        17 -197179.88736731    +499.53301285
        18 -196696.01860862    +483.86875869
        19 -196215.74967921    +480.26892941
        20 -195814.45216928    +401.29750993
        21 -195467.19689949    +347.25526979
        22 -195177.88536020    +289.31153929
        23

In [ ]:
#Sampling a sequence of music from the model and save as a MIDI file, name it "event_based.mid"

generated_events = generated_sequence.flatten().astype(int)


synthetic_music = muspy.from_event_representation(
    generated_events,
    resolution=24,
    program=0,
    is_drum=False,
    use_single_note_off_event=False,
    use_end_of_sequence_event=False,
    max_time_shift=100,
    velocity_bins=32,
    default_velocity=64,
    duplicate_note_mode='fifo'
)

# Save as MIDI file
muspy.write_midi('/content/drive/MyDrive/GenAi/PA2.2_dataset/event_based.midi', synthetic_music)


To add some fun, lets take it up a notch and go for the __Note Based Representation__.  
More on that here:
1. https://muspy.readthedocs.io/en/v0.3.0/representations/note.html
2. https://salu133445.github.io/muspy/classes/note.html

**Hint:** This is a bit tricky since we have 4 features per observation. We'll leave it to you to devise a way to deal with it.  \
There are alot of approaches which can be used. As some features are categorical, and some are continuous, hence you can try different HMMs types or a single HMM to rule them all. Just generate some good music. \
Before you start, do take a peek of the available HMM models in the HMMlearn library __(you are allowed to import additional models if you want)__

In [ ]:
#Write your code here and save the sampled sequence as MIDI file, name it "note_based.mid"
note_based_collection = [muspy.to_note_representation(music, encode_velocity=True) for music in my_dataset]
X = np.concatenate([np.array(music) for music in note_based_collection])


In [ ]:
continuous_features = []  # For time, duration, velocity
discrete_features = []    # For pitch

for music in note_based_collection:
    # Assuming music is a N x 4 array: columns are [time, pitch, duration, velocity]
    continuous_data = music[:, [0, 2, 3]]  # Selecting time, duration, velocity
    pitches = music[:, 1]  # Pitch is the second column

    continuous_features.append(continuous_data)
    discrete_features.append(pitches)

X_continuous = np.concatenate(continuous_features).reshape(-1, 3)  # 3 continuous features
X_discrete = np.concatenate(discrete_features).reshape(-1, 1)
lengths = [len(music) for music in note_based_collection]

continuous_model = hmm.GaussianHMM(n_components=5, n_iter=100, random_state=42,verbose=True)
continuous_model.fit(X_continuous, lengths)



         1 -262138.04295325             +nan
         2 -178282.20293057  +83855.84002268
         3 -171238.22702849   +7043.97590208
         4 -166600.43893633   +4637.78809216
         5 -164633.21585829   +1967.22307804
         6 -164378.24483998    +254.97101831
         7 -164347.63635833     +30.60848165
         8 -164343.02066796      +4.61569038
         9 -164340.88342502      +2.13724294
        10 -164339.43457096      +1.44885405
        11 -164338.50498083      +0.92959013
        12 -164337.83377561      +0.67120523
        13 -164337.28773991      +0.54603570
        14 -164336.95476054      +0.33297937
        15 -164336.65048398      +0.30427656
        16 -164336.39892074      +0.25156324
        17 -164336.18850650      +0.21041424
        18 -164336.00437300      +0.18413351
        19 -164335.81965667      +0.18471633
        20 -164335.63990141      +0.17975526
        21 -164335.45531701      +0.18458439
        22 -164335.29392073      +0.16139629
        23

GaussianHMM(n_components=5, n_iter=100, random_state=42, verbose=True)

In [ ]:
discrete_model = CategoricalHMM(n_components=8, n_iter=100,random_state=42, verbose=True)
discrete_model.fit(X_discrete, lengths)


         1 -107300.36336950             +nan
         2  -84009.48833794  +23290.87503156
         3  -83586.79680990    +422.69152804
         4  -83199.75172469    +387.04508520
         5  -82826.14301072    +373.60871397
         6  -82327.96751371    +498.17549701
         7  -81624.64003851    +703.32747520
         8  -80782.34861834    +842.29142017
         9  -79965.50481651    +816.84380182
        10  -79391.94307966    +573.56173685
        11  -79007.92357017    +384.01950949
        12  -78788.48832894    +219.43524123
        13  -78644.99218133    +143.49614762
        14  -78543.27963819    +101.71254314
        15  -78466.99860697     +76.28103122
        16  -78401.30317728     +65.69542969
        17  -78340.93614415     +60.36703313
        18  -78283.16031667     +57.77582747
        19  -78220.77870002     +62.38161666
        20  -78159.65984474     +61.11885528
        21  -78098.39852639     +61.26131835
        22  -78007.37155353     +91.02697285
        23

CategoricalHMM(n_components=8, n_features=97, n_iter=100,
               random_state=RandomState(MT19937) at 0x7C59E01B1A40,
               verbose=True)

In [ ]:
#print(pitches)
print("Unique pitches:", np.unique(X_discrete))
print(X_continuous)

Unique pitches: [36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89 90 91 93 95 96]
[[   0   12   64]
 [   0   24   64]
 [   0   24   64]
 ...
 [2952   48   64]
 [2952   48   64]
 [2952   72   64]]


In [ ]:
# Generate a sequence from the continuous model
generated_continuous, _ = continuous_model.sample(synthetic_music_sequence_length)

# Generate a sequence from the discrete model
generated_discrete, _ = discrete_model.sample(synthetic_music_sequence_length)


In [ ]:
combined_sequence = np.hstack((
    generated_continuous[:, 0:1],  # Time
    generated_discrete,            # Pitch
    generated_continuous[:, 1:2],  # Duration
    generated_continuous[:, 2:3]   # Velocity
))
combined_sequence = np.round(combined_sequence).astype(int)


In [ ]:
# Calculate delta times
delta_times = np.diff(combined_sequence[:, 0], prepend=0)

# Ensure delta times are non-negative
delta_times = np.maximum(delta_times, 0)

# Replace the time column in the combined sequence with delta times
combined_sequence[:, 0] = delta_times
# Ensure pitch, duration, and velocity are within MIDI range and are integers
combined_sequence[:, 1] = np.clip(combined_sequence[:, 1], 0, 127).astype(int)  # Pitch
combined_sequence[:, 2] = np.clip(combined_sequence[:, 2], 1, np.inf).astype(int)  # Duration
combined_sequence[:, 3] = np.clip(combined_sequence[:, 3], 0, 127).astype(int)  # Velocity


In [ ]:
synthetic_music = muspy.from_note_representation(
    combined_sequence,
    resolution=24,
    program=0,
    is_drum=False,
    use_start_end=True,
    encode_velocity=True,
    default_velocity=64
)

# Save as MIDI file
muspy.write_midi('/content/drive/MyDrive/GenAi/PA2.2_dataset/notes_based.midi', synthetic_music)